In [1]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
df = df.iloc[:300]

In [3]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
q = df.iloc[0].answer_llm

In [5]:
v = embedding_model.encode(q)

In [6]:
v[0]

np.float32(-0.42244655)

In [7]:
records = df.to_dict(orient='records')

In [8]:
from tqdm.auto import tqdm

In [9]:
evaluations = []

for r in tqdm(records):
    u = embedding_model.encode(r['answer_llm'])
    v = embedding_model.encode(r['answer_orig'])
    evaluations.append(u.dot(v))

  0%|          | 0/300 [00:00<?, ?it/s]

In [10]:
evaluations = pd.Series(evaluations)
evaluations.describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
dtype: float64

In [11]:
u.dot(u)

np.float32(36.947296)

In [12]:
import numpy as np

In [13]:
def normalize(v):
    return v / np.sqrt((v * v).sum())

In [16]:
vn=normalize(v)
vn.dot(vn)

np.float32(1.0)

In [17]:
evaluations = []

for r in tqdm(records):
    u = embedding_model.encode(r['answer_llm'])
    v = embedding_model.encode(r['answer_orig'])
    u = normalize(u)
    v = normalize(v)
    evaluations.append(u.dot(v))

  0%|          | 0/300 [00:00<?, ?it/s]

In [18]:
evaluations = pd.Series(evaluations)
evaluations.describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
dtype: float64

In [19]:
from rouge import Rouge
rouge_scorer = Rouge()

In [20]:
r = records[10]

In [21]:
print(r)

{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.", 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.', 'document': '5170565b', 'question': 'Are sessions recorded if I miss one?', 'course': 'machine-learning-zoomcamp'}


In [22]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [28]:
scores['rouge-1']['f']

0.45454544954545456

In [29]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [30]:
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [31]:
rouge_avg

0.35490034990035496

In [32]:
evaluations = []

for r in tqdm(records):
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    
    evaluations.append({
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'rouge_avg': (rouge_1 + rouge_2 + rouge_l) / 3
    })

  0%|          | 0/300 [00:00<?, ?it/s]

In [33]:
pd.DataFrame(evaluations).rouge_2.mean()

np.float64(0.20696501983423318)